In [278]:
# __original_author__ =  'Humberto Brandão'
#__tweak__ = 'Team Monami'
import pandas as pd
import xgboost as xgb
import pickle as pkl
import numpy as np

# from matplotlib import pylab as plt
# import operator

# Load data and roughly clean it, then sort as game date
with open('df_test.pkl', 'rb') as f:
    df = pkl.load(f)
    
# df.drop(['game_event_id', 'game_id', 'lat', 'lon', 'team_id', 'team_name'], axis=1, inplace=True)
# df.sort_values('game_date',  inplace=True)
mask = np.random.rand(len(df)) < 0.2 #mask for training set

In [280]:
df.index

Index(['freeboard_1377351', 'freeboard_1309879', 'freeboard_1331187',
       'freeboard_1255886', 'freeboard_1240142', 'freeboard_1346887',
       'freeboard_1345544', 'freeboard_1256458', 'freeboard_1249492',
       'freeboard_1341092',
       ...
       'star_377869', 'star_351447', 'star_355807', 'star_359453',
       'star_357915', 'star_369784', 'star_351841', 'star_385072',
       'star_388428', 'star_379388'],
      dtype='object', name='contentID', length=45471)

In [281]:
# Clean data
# actiontypes = dict(df.action_type.value_counts())
# df['type'] = df.apply(lambda row: row['action_type'] if actiontypes[row['action_type']] > 25 else 'others', axis=1)
# df.drop(['action_type', 'combined_shot_type'], axis=1, inplace=True)

# df['away'] = df.matchup.str.contains('@')
# df.drop('matchup', axis=1, inplace=True)

# df['distance'] = df.apply(lambda row: row['shot_distance'] if row['shot_distance'] <45 else 45, axis=1)
# df['time_remaining'] = df.apply(lambda row: row['minutes_remaining'] * 60 + row['seconds_remaining'], axis=1)
# df['last_moments'] = df.apply(lambda row: 1 if row['time_remaining'] < 3 else 0, axis=1)

# data = pd.get_dummies(df['type'],prefix="action_type")

# features=["away", "period", "playoffs", "shot_type", "shot_zone_area", "shot_zone_basic", "season",
#            "shot_zone_range", "opponent", "last_moments"]
# for f in features:
#     data = pd.concat([data, pd.get_dummies(df[f], prefix=f),], axis=1)
# data = pd.concat([data, df['distance'],], axis=1)

features = ['normalPostCount', 'bestPostCount', 'BOBPostCount', 'okIn900s', 'memoIn900s', 'imgCount', 'youtubeCount', 'videoCount', 
            'writerVisitingCount','prohibitBOBOption', 'textLineCount']
features += ['morning', 'noon', 'evening', 'night']

data = df[features].copy()
data.rename(columns=lambda x: x.replace(" ",""),inplace=True)

X = data[~mask]
y = df.isBOB[~mask]
# y = df.viewCount[~mask]

dtrain=xgb.DMatrix(data=X, label=y, missing =None)

In [286]:
param ={   'objective'             : "binary:logistic",
                'booster'               : "gbtree",
                'eval_metric'          : "logloss",
                'eta'                      : 0.012,
                'max_depth'           : 10,
                'subsample'           : 0.82,
                'colsample_bytree'  : 0.60,
                'seed'                    : 1,
                'silent'                   : 0
}
clf = xgb.cv(   params               = param,
                dtrain                      = dtrain,
                num_boost_round    = 1000,
                maximize                 = False,
                nfold                       = 5,
                early_stopping_rounds  = 20
);
print(clf[[0]].min(0))
bestround = clf[[0]].idxmin(0)[0]

KeyboardInterrupt: 

In [ ]:
# param ={   'objective'                : "reg:linear",
#                 'booster'             : "gbtree",
#                 'eval_metric'         : "rmse",
#                 'eta'                 : 0.012,
#                 'max_depth'           : 10,
#                 'subsample'           : 0.82,
#                 'colsample_bytree'    : 0.60,
#                 'seed'                : 1,
#                 'silent'              : 0
# }
# clf = xgb.cv(   params               = param,
#                 dtrain                      = dtrain,
#                 num_boost_round    = 1000,
#                 maximize                 = False,
#                 nfold                       = 5,
#                 early_stopping_rounds  = 20
# );
# print(clf[[0]].min(0))
# bestround = clf[[0]].idxmin(0)[0]

In [ ]:
bst = xgb.train(  params                    = param,
                        dtrain                       = dtrain,
                        num_boost_round     = bestround,
                        verbose_eval            = 1,
                        maximize                  = False
)

In [ ]:
test_X = data[mask]
test_y = df[mask]["isBOB"]
dtest =xgb.DMatrix(data=test_X, missing =None)
preds = bst.predict(dtest)
# submission = pd.DataFrame({"shot_id": target_id, "shot_made_flag": preds})
# submission = submission.sort_values('shot_id')
# submission.to_csv("submit_xgboost_ref.csv", index=False)


# def ceate_feature_map(features):
    # outfile = open('xgb.fmap', 'w')
    # i = 0
    # for feat in features:
        # outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        # i = i + 1
    # outfile.close()

# ceate_feature_map(list(X.columns))
# # ceate_feature_map(features)
# importance = bst.get_fscore(fmap='xgb.fmap')
# importance = sorted(importance.items(), key=operator.itemgetter(1))

# df = pd.DataFrame(importance, columns=['feature', 'fscore'])
# df['fscore'] = df['fscore'] / df['fscore'].sum()

# plt.figure()
# df.plot()
# df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(25, 30))
# plt.title('XGBoost Feature Importance')
# plt.xlabel('relative importance')
# plt.gcf().savefig('feature_importance_xgb.png')

In [ ]:
from  sklearn.metrics import confusion_matrix
confusion_matrix(test_y.values, preds > 0.5)

In [ ]:
(p[0][0] + p[1][1])/ np.sum(p)

array([[6788,  338],
       [ 900, 1127]])

In [277]:
########  SVM
from sklearn import svm
clf = svm.SVC()

clf.fit(X,y)

score = clf.score(test_X,test_y)
y_predict = clf.predict(test_X)
y_All_no = np.zeros(len(test_y))
print(confusion_matrix(test_y, y_predict), score)
print(confusion_matrix(y_All_no, y_predict))
#df_all_new = pd.concat([df_all,df_all_timeZone], axis = 1)

[[3956   15]
 [ 203   75]] 0.948693810308
[[4159   90]
 [   0    0]]
